In [ ]:
!pip install numpy==1.23.5

In [ ]:
!pip install lightgbm

In [ ]:
!pip install arff

In [1]:
from os.path import join
def printf(*args, fname="log.txt"):
    with open(join("test_outputs",fname),"a+") as f:
        for a in args:
            f.write(str(a) + " ")
        f.write("\n") 
    print(args) 

# KDD 98

In [2]:
#!mkdir -p lifetime-value/kdd_cup_98
#!wget https://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98lrn.zip -P lifetime-value/kdd_cup_98/
#!wget https://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/cup98val.zip -P lifetime-value/kdd_cup_98/
#!wget https://kdd.ics.uci.edu/databases/kddcup98/epsilon_mirror/valtargt.txt -P lifetime-value/kdd_cup_98/


In [3]:
#!unzip lifetime-value/kdd_cup_98/cup98lrn.zip
#!unzip lifetime-value/kdd_cup_98/cup98val.zip

In [4]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('cup98LRN.txt')
num_train = df_train.shape[0]
df_eval = pd.read_csv('cup98VAL.txt')
df_eval_target = pd.read_csv('lifetime-value/kdd_cup_98/valtargt.txt')
df_eval = df_eval.merge(df_eval_target, on='CONTROLN')

/tmp/ipykernel_62563/3556400376.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('cup98LRN.txt')
/tmp/ipykernel_62563/3556400376.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_eval = pd.read_csv('cup98VAL.txt')


In [5]:
# Original: https://github.com/google/lifetime_value/blob/master/notebooks/kdd_cup_98/regression.ipynb
df = pd.concat([df_train, df_eval], axis=0, sort=True)
y = df['TARGET_D'][:num_train]

VOCAB_FEATURES = [
    'ODATEDW',  # date of donor's first gift (YYMM)
    'OSOURCE',  # donor acquisition mailing list
    'TCODE',    # donor title code
    'STATE',
    'ZIP',
    'DOMAIN',   # urbanicity level and socio-economic status of the neighborhood
    'CLUSTER',  # socio-economic status
    'GENDER',
    'MAXADATE', # date of the most recent promotion received
    'MINRDATE',
    'LASTDATE',
    'FISTDATE',
    'RFA_2A',
]

df['ODATEDW'] = df['ODATEDW'].astype('str')
df['TCODE'] = df['TCODE'].apply(
    lambda x: '{:03d}'.format(x // 1000 if x > 1000 else x))
df['ZIP'] = df['ZIP'].str.slice(0, 5)
df['MAXADATE'] = df['MAXADATE'].astype('str')
df['MINRDATE'] = df['MINRDATE'].astype('str')
df['LASTDATE'] = df['LASTDATE'].astype('str')
df['FISTDATE'] = df['FISTDATE'].astype('str')


def label_encoding(y, frequency_threshold=100):
  value_counts = pd.value_counts(y)
  categories = value_counts[
      value_counts >= frequency_threshold].index.to_numpy()
  # 0 indicates the unknown category.
  return pd.Categorical(y, categories=categories).codes + 1

for key in VOCAB_FEATURES:
  df[key] = label_encoding(df[key])

MAIL_ORDER_RESPONSES = [
    'MBCRAFT',
    'MBGARDEN',
    'MBBOOKS',
    'MBCOLECT',
    'MAGFAML',
    'MAGFEM',
    'MAGMALE',
    'PUBGARDN',
    'PUBCULIN',
    'PUBHLTH',
    'PUBDOITY',
    'PUBNEWFN',
    'PUBPHOTO',
    'PUBOPP',
    'RFA_2F',
]

INDICATOR_FEATURES = [
    'AGE',  # age decile, 0 indicates unknown
    'NUMCHLD',
    'INCOME',
    'WEALTH1',
    'HIT',
] + MAIL_ORDER_RESPONSES

df['AGE'] = pd.qcut(df['AGE'].values, 10).codes + 1
df['NUMCHLD'] = df['NUMCHLD'].apply(lambda x: 0 if np.isnan(x) else int(x))
df['INCOME'] = df['INCOME'].apply(lambda x: 0 if np.isnan(x) else int(x))
df['WEALTH1'] = df['WEALTH1'].apply(lambda x: 0 if np.isnan(x) else int(x) + 1)
df['HIT'] = pd.qcut(df['HIT'].values, q=50, duplicates='drop').codes

for col in MAIL_ORDER_RESPONSES:
  df[col] = pd.qcut(df[col].values, q=20, duplicates='drop').codes + 1

NUMERIC_FEATURES = [
    # binary
    'MAILCODE',  # bad address
    'NOEXCH',    # do not exchange
    'RECINHSE',  # donor has given to PVA's in house program
    'RECP3',     # donor has given to PVA's P3 program
    'RECPGVG',   # planned giving record
    'RECSWEEP',  # sweepstakes record
    'HOMEOWNR',  # home owner
    'CHILD03',
    'CHILD07',
    'CHILD12',
    'CHILD18',

    # continuous
    'CARDPROM',
    'NUMPROM',
    'CARDPM12',
    'NUMPRM12',
    'RAMNTALL',
    'NGIFTALL',
    'MINRAMNT',
    'MAXRAMNT',
    'LASTGIFT',
    'AVGGIFT',
]
     

df['MAILCODE'] = (df['MAILCODE'] == 'B').astype('float32')
df['PVASTATE'] = df['PVASTATE'].isin(['P', 'E']).astype('float32')
df['NOEXCH'] = df['NOEXCH'].isin(['X', '1']).astype('float32')
df['RECINHSE'] = (df['RECINHSE'] == 'X').astype('float32')
df['RECP3'] = (df['RECP3'] == 'X').astype('float32')
df['RECPGVG'] = (df['RECPGVG'] == 'X').astype('float32')
df['RECSWEEP'] = (df['RECSWEEP'] == 'X').astype('float32')
df['HOMEOWNR'] = (df['HOMEOWNR'] == 'H').astype('float32')
df['CHILD03'] = df['CHILD03'].isin(['M', 'F', 'B']).astype('float32')
df['CHILD07'] = df['CHILD07'].isin(['M', 'F', 'B']).astype('float32')
df['CHILD12'] = df['CHILD12'].isin(['M', 'F', 'B']).astype('float32')
df['CHILD18'] = df['CHILD18'].isin(['M', 'F', 'B']).astype('float32')

df['CARDPROM'] = df['CARDPROM'] / 100
df['NUMPROM'] = df['NUMPROM'] / 100
df['CARDPM12'] = df['CARDPM12'] / 100
df['NUMPRM12'] = df['NUMPRM12'] / 100
df['RAMNTALL'] = np.log1p(df['RAMNTALL'])
df['NGIFTALL'] = np.log1p(df['NGIFTALL'])
df['MINRAMNT'] = np.log1p(df['MINRAMNT'])
df['MAXRAMNT'] = np.log1p(df['MAXRAMNT'])
df['LASTGIFT'] = np.log1p(df['LASTGIFT'])
df['AVGGIFT'] = np.log1p(df['AVGGIFT'])

CATEGORICAL_FEATURES = VOCAB_FEATURES + INDICATOR_FEATURES
ALL_FEATURES = CATEGORICAL_FEATURES + NUMERIC_FEATURES

def dnn_split(df):
  df_train = df.iloc[:num_train]
  df_eval = df.iloc[num_train:]

  def feature_dict(df):
    features = {k: v.values.reshape(-1,1) for k, v in dict(df[CATEGORICAL_FEATURES]).items()}
    features['numeric'] = df[NUMERIC_FEATURES].astype('float32').values
    return features

  x_train, y_train = feature_dict(df_train), df_train['TARGET_D'].astype(
      'float32').values
  x_eval, y_eval = feature_dict(df_eval), df_eval['TARGET_D'].astype(
      'float32').values

  return x_train, x_eval, y_train, y_eval

/tmp/ipykernel_62563/2921462118.py:32: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  value_counts = pd.value_counts(y)
/tmp/ipykernel_62563/2921462118.py:32: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  value_counts = pd.value_counts(y)
/tmp/ipykernel_62563/2921462118.py:32: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  value_counts = pd.value_counts(y)
/tmp/ipykernel_62563/2921462118.py:32: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  value_counts = pd.value_counts(y)
/tmp/ipykernel_62563/2921462118.py:32: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() inste

In [6]:
x_train, x_eval, y_train, y_eval = dnn_split(df)

In [7]:
all_data = [{0:[np.hstack(list(x_train.values())),y_train,np.hstack(list(x_eval.values())),y_eval]}]

In [8]:
import numpy as np

from sklearn import datasets

# Diabetes

In [9]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [10]:
all_data.append({0:[X,y]})


# California housing

In [11]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
all_data.append({0:[X.to_numpy(),y.to_numpy()]})

# Liver disorders

In [12]:
from sklearn.datasets import fetch_openml
ld = fetch_openml(name='liver-disorders')

In [13]:
X, y = ld['data'].to_numpy(),ld['target'].to_numpy()
all_data.append({0:[X,y]})

In [14]:

from sklearn.model_selection import train_test_split
dict_data = {}
dict_data["Diabetes"] = {}
dict_data["California housing"] = {}
dict_data["Liver disorders"] = {}
dict_data["KDD98"] = {}

for k in all_data[0]:
    x01,x02,y01,y02 = all_data[0][k][0], all_data[0][k][2], all_data[0][k][1], all_data[0][k][3]
    dict_data["KDD98"][k] = {"train":{"X":x01,"y":y01},"test":{"X":x02,"y":y02}}

for k in all_data[1]:
    x01,x02,y01,y02 = train_test_split(all_data[1][k][0], all_data[1][k][1], test_size=0.3,random_state=42)
    dict_data["Diabetes"][k] = {"train":{"X":x01,"y":y01},"test":{"X":x02,"y":y02}}
    
for k in all_data[2]:
    x11,x12,y11,y12 = train_test_split(all_data[2][k][0], all_data[2][k][1],test_size=0.3,random_state=42)
    dict_data["California housing"][k] = {"train":{"X":x11,"y":y11},"test":{"X":x12,"y":y12}} 

for k in all_data[3]:
    x11,x12,y11,y12 = train_test_split(all_data[3][k][0], all_data[3][k][1],test_size=0.3,random_state=42)
    dict_data["Liver disorders"][k] = {"train":{"X":x11,"y":y11},"test":{"X":x12,"y":y12}} 

all_data = dict_data

In [15]:
all_data["KDD98"][0]["train"]["X"].shape

(95412, 54)

In [16]:
all_data["KDD98"][0]["train"]["y"].shape

(95412,)

In [17]:
all_data["KDD98"][0]["test"]["y"].shape

(96367,)

In [18]:
all_data["KDD98"][0]["test"]["X"].shape

(96367, 54)

# Boosting

In [19]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

np.bool = np.bool_

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelXGB(max_depth,layers,C,n_trees):
    return xgb.XGBRegressor(max_depth = max_depth, n_estimators = layers)

def make_modelCascade(max_depth,layers,C,n_trees):
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4,backend="sklearn",criterion='squared_error',n_trees=n_trees)

def make_modelBoosted(max_depth,layers,C,hs,n_trees):
    return CascadeBoostingRegressor(C=C, n_layers=layers, n_estimators = 4, max_depth=max_depth, n_iter_no_change = 1, validation_fraction = 0.1, learning_rate = 0.9,hidden_size = hs,n_trees=n_trees)


#models = {"XGB":make_modelXGB,"Cascade Forest":make_modelCascade, "Boosted Forest": make_modelBoosted}
models = {"Boosted Forest": make_modelBoosted}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in ["KDD98"]:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 1, 10)
                max_depth = trial.suggest_int('max_depth', 1, 2)

                if model_name == "Boosted Forest":
                    C = trial.suggest_int('C', 10, 2000)
                    hs = trial.suggest_int('hs', 2, 8)
                    n_trees = trial.suggest_int('trees', 10, 100)
                else:
                    if model_name == "Cascade Forest":
                        n_trees = trial.suggest_int('trees', 10, 100)
                    else:
                        n_trees = 0    
                    C = 0
                    hs = 0

                scores = []
                for _ in range(3):
                    kf = KFold(n_splits=5)
                    for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                        if hs > 0:
                            model = make_model(max_depth,layers,C,hs,n_trees)
                        else:    
                            model = make_model(max_depth,layers,C,n_trees)
                        
                        model.fit(
                             x_train[train_index],
                             Y_train[train_index],
                        )
                        y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                        scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)    
            
            layers = 2#study.best_trial.params["layers"]  
            max_depth = 1#study.best_trial.params["max_depth"]  

            if model_name == "Boosted Forest":
                C = study.best_trial.params["C"]
                hs = study.best_trial.params["hs"]
                n_trees = study.best_trial.params["trees"]
            else:
                if model_name == "Cascade Forest":
                    n_trees = study.best_trial.params["trees"]
                else:
                    n_trees = 0
                C = 0
                hs = 0
            if hs > 0:    
                model = make_model(max_depth,layers,C,hs,n_trees)
            else:
                model = make_model(max_depth,layers,C,n_trees)
                
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),fname="classic_datasets/boosting_output.txt")     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-05-16 22:45:50,869] A new study created in memory with name: no-name-2587f21b-d027-4411-a002-489e655160ad
/usr/local/lib/python3.9/dist-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
[W 2025-05-16 22:46:54,336] Trial 0 failed with parameters: {'layers': 7, 'max_depth': 2, 'C': 1121, 'hs': 2, 'trees': 56} because of the following error: UnboundLocalError("local variable 'train_loader' referenced before 

UnboundLocalError: local variable 'train_loader' referenced before assignment

In [ ]:
x_train.shape

# Build a grid-map

In [ ]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

np.bool = np.bool_

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelXGB(max_depth,layers,C):
    return xgb.XGBRegressor(max_depth = max_depth, n_estimators = layers)

def make_modelCascade(max_depth,layers,C):
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4,backend="sklearn",criterion='squared_error')

def make_modelBoosted(max_depth,layers,C,hs):
    return CascadeBoostingRegressor(C=C, n_layers=layers, n_estimators = 4, max_depth=max_depth, n_iter_no_change = 1, validation_fraction = 0.1, learning_rate = 0.9,hidden_size = hs)


#models = {"XGB":make_modelXGB,"Cascade Forest":make_modelCascade, "Boosted Forest": make_modelBoosted}
models = {"Boosted Forest": make_modelBoosted}

bo_data = {}    

for model_name in models:
    make_model = models[model_name]
    for ds_name in ["Diabetes"]:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            for layers in [1,2,3,4,5]:
                for hs in [1,2,3,4,5,6,7,8]:
                    scores = []
                    scores2 = []
                    for _ in range(3):
                        max_depth = 1 #trial.suggest_int('max_depth', 1, 2)
                        C = 100 #trial.suggest_int('C', 10, 2000)
                        kf = KFold(n_splits=5)
                        
                        for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                            if hs > 0:
                                model = make_model(max_depth,layers,C,hs)
                            else:    
                                model = make_model(max_depth,layers,C)
                            
                            model.fit(
                                 x_train[train_index],
                                 Y_train[train_index],
                            )
                            y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                            y_pred2 = model.predict(x_test) #, batch_size=batch_size)
                            scores.append([mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten())])
                            scores2.append([mean_squared_error(Y_test.flatten(),y_pred2.flatten())])
                    bo_data[(layers,hs)] = [np.asarray(scores).mean(),np.asarray(scores2).mean()]
    

In [ ]:
bo_data

In [ ]:
import pickle

with open('bo_data.pickle', 'wb') as handle:
    pickle.dump(bo_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
!nvidia-smi

# Alternative weighting

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from ecdfr.gcForest import gcForest
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelECDFR(max_depth,layers,resampling_rate, et):
    config = {"estimator_configs":[{"n_fold": 5,"type":None,"max_depth":max_depth}],
              "error_threshold": et,
              "resampling_rate": resampling_rate,
              "random_state":None,
              "max_layers":layers,
              "early_stop_rounds":1,
              "train_evaluation":mean_squared_error}
    
    return gcForest(config,1)

models = {"ecdfr":make_modelECDFR}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 5, 15)
                max_depth = trial.suggest_int('max_depth', 1, 2)

                C = trial.suggest_float('resampling_rate', 1.1, 1.8)
                et = trial.suggest_float('et', 0.01, 0.9)
                
                kf = KFold(n_splits=3)
                scores = []
                try:
                    for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                        model = make_modelECDFR(max_depth,layers,C,et)
                    
                        model.fit(
                             x_train[train_index],
                             Y_train[train_index],
                        )
                        y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                        scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                except:
                    scores = [1000000000.]
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=1000)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  


            C = study.best_trial.params["resampling_rate"]  
            et = study.best_trial.params["et"]  
            model = make_model(max_depth,layers,C,et)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            print(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max())     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])

In [ ]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelCascade(max_depth,layers,C,wt):
    wf = {0:"linear", 1:"1-w^1/2", 2:"1-w2"}
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4,adaptive=True,weighting_function = wf[wt])


models = {"AWDF":make_modelCascade}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 5, 15)
                max_depth = trial.suggest_int('max_depth', 1, 2)
                wt = trial.suggest_int('weight_function', 0, 2)   
                if model_name == "Boosted Forest":
                    C = trial.suggest_int('C', 1, 2000)
                else:
                    C = 0

                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(max_depth,layers,C,wt)
                    
                    model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                    )
                    y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                    scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=50)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  
            wt = study.best_trial.params['weight_function']
            if model_name == "Boosted Forest":
                C = study.best_trial.params["C"]  
            else:
                C = 0
            model = make_model(max_depth,layers,C,wt)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),fname="classic_datasets/awdf_output.txt")     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    